## Read lick events from openephys events folder

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path
from datetime import datetime
import dateutil

In [ ]:
file = '/data/Clustering/SleepDeprivation/RatS/Day2NSD/lick_events/'
p = Path(file)
folders = [x for x in p.iterdir() if x.is_dir()]

for folder in folders:
    time = dateutil.parser.parse(folder.name,fuzzy=True)
    print(time)

## Create channel map for openephys in json format

In [2]:
import json
import numpy as np
import pandas as pd
import subjects

In [3]:
sess = subjects.Sd().ratNday1[0]
auxchans = sess.recinfo.auxchans
chans = np.concatenate(sess.recinfo.channelgroups)
chans = np.concatenate((chans,chans+128,auxchans+128,auxchans+128+6))
chan_map_oe = {"0":{"mapping":[int(_+1) for _ in chans],"reference":[-1]*len(chans),"enabled":['true']*len(chans)},"refs":{"channels":[]},"recording":{"channels":[]}}
with open('test_chanmap','w') as jfile:
    json.dump(chan_map_oe,jfile,indent=4)


## Testing ephyviewer for viewing ephys signal

In [9]:
from ephyviewer import mkQApp, MainViewer, TraceViewer
import numpy as np
import subjects

#you must first create a main Qt application (for event loop)
app = mkQApp()

#-------signals -----------
sess = subjects.Sd().ratNday1[0]
maze = sess.epochs.maze
changrp = sess.recinfo.channelgroups[0]
sigs = np.asarray(sess.recinfo.geteeg(chans=changrp,timeRange=maze))
# sigs = sigs[:,np.newaxis]
sigs = sigs.T
sample_rate = float(sess.recinfo.lfpSrate)
print('done')
#create fake 16 signals with 100000 at 10kHz
# sigs = np.random.rand(400000,16)
# sample_rate = 1250
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
# TraceViewer normally accept a AnalogSignalSource but
# TraceViewer.from_numpy is facitilty function to bypass that
view1 = TraceViewer.from_numpy(sigs, sample_rate, t_start,'Signals')

#Parameters can be set in script
view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
view1.params

#And also parameters for each channel
# view1.by_channel_params['ch0', 'visible'] = False
# view1.by_channel_params['ch15', 'color'] = '#FF00AA'

#This is needed when scale_mode='same_for_all'
#to recompute the gain
#this avoid to push auto_scale button
view1.auto_scale()

#put this veiwer in the main window
win.add_view(view1)
#show main window and run Qapp
win.show()

app.exec_()


done
debug True
refresh duration for Signals 0.0001709461212158203 s
save_all_settings


0

## Testing sleep states editor

In [1]:
"""
ephyviewer also provides an epoch encoder which can be used with shortcut keys
and/or the mouse to encode labels.

ephyviewer makes available a CsvEpochSource class, which inherits from
WritableEpochSource. If you would like to customize reading and writing epochs
to files, you can write your own subclass of WritableEpochSource that implements
the load() and save() methods.

Here is an example of an epoch encoder that uses CsvEpochSource.

"""

from ephyviewer import mkQApp, MainViewer, TraceViewer, CsvEpochSource, EpochViewer,WritableEpochSource, EpochEncoder, SpikeTrainViewer, TimeFreqViewer
from ephyviewer import InMemoryEpochSource, InMemorySpikeSource, InMemoryAnalogSignalSource
import numpy as np
import subjects
import os
import pandas as pd
import signal_process

class statesSource(WritableEpochSource):
    def __init__(self, filename, possible_labels, color_labels=None, channel_name='', restrict_to_possible_labels=False):

        self.filename = filename

        WritableEpochSource.__init__(self, epoch=None, possible_labels=possible_labels, color_labels=color_labels, channel_name=channel_name, restrict_to_possible_labels=restrict_to_possible_labels)

    def load(self):
        """
        Returns a dictionary containing the data for an epoch.
        Data is loaded from the CSV file if it exists; otherwise the superclass
        implementation in WritableEpochSource.load() is called to create an
        empty dictionary with the correct keys and types.
        The method returns a dictionary containing the loaded data in this form:
        { 'time': np.array, 'duration': np.array, 'label': np.array, 'name': string }
        """

        if os.path.exists(self.filename):
            # if file already exists, load previous epoch
            data = pd.read_pickle(self.filename)
            df = pd.DataFrame()
            df['time']= data['start']
            df['duration'] = data['end']-data['start']
            df['label'] = 'U'
            # df = pd.read_csv(self.filename,  index_col=None, dtype={
            #     'time':     'float64',
            #     'duration': 'float64',
            #     'label':    'U'})
            state_number_dict = {
                1: "nrem",
                2: "rem",
                3: "quiet",
                4: "active",
            }
            data["name"] = data["state"].map(state_number_dict)

            epoch_labels = np.array([f' State{_}' for _ in data['state']])
            epoch = {'time':     data['start'].values,
                     'duration': data['end'].values-data['start'].values,
                     'label':    data['name'],
                     'name':     '128'}
        else:
            # if file does NOT already exist, use superclass method for creating
            # an empty dictionary
            epoch = super().load()

        return epoch

    def save(self):
        df = pd.DataFrame()
        df['time'] = np.round(self.ep_times, 6)         # round to nearest microsecond
        df['duration'] = np.round(self.ep_durations, 6) # round to nearest microsecond
        df['label'] = self.ep_labels
        df.sort_values(['time', 'duration', 'label'], inplace=True)
        df.to_pickle(self.filename)

sess = subjects.Sd().ratNday1[0]
# sleep_states = sess.brainstates.states
filename = sess.brainstates.files.states


# all_epochs = [{'time':sleep_states.start.values,'duration':sleep_states.duration.values,'label':sleep_states.name.to_numpy(),'name':'nrem'}]
# source_ep = InMemoryEpochSource(all_epochs=all_epochs)
possible_labels = ['nrem', 'rem', 'quiet',  'active']
source_epoch = statesSource(str(filename),possible_labels)
# lets encode some dev mood along the day

# filename = 'example_dev_mood_encoder.csv'
# source_epoch = CsvEpochSource(filename, possible_labels)


#you must first create a main Qt application (for event loop)
app = mkQApp()

#create fake 16 signals with 100000 at 10kHz
sigs = np.asarray(sess.recinfo.geteeg(chans = 56)).reshape(-1,1)
filtered_sig = signal_process.filter_sig.bandpass(sigs,lf=120,hf=150,ax=0,fs=1250)
sample_rate = sess.recinfo.lfpSrate
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal
view1 = TraceViewer.from_numpy(np.hstack(sigs,filtered_sig), sample_rate, t_start, 'Signals')
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
win.add_view(view1)

source_sig = InMemoryAnalogSignalSource(sigs,sample_rate,t_start)
#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Dev mood states along day')
win.add_view(view2)

view3 = TimeFreqViewer(source=source_sig,name='tfr')
view3.params['show_axis'] = False
view3.params['timefreq', 'deltafreq'] = 1
win.add_view(view3)


#----- spikes --------
spikes = sess.spikes.pyr
spk_id = sess.spikes.pyrid

all_spikes=[]
for i,(t,id_) in enumerate(zip(spikes,spk_id)):
    all_spikes.append({'time': t,'name': f"Unit {i}"})

spike_source = InMemorySpikeSource(all_spikes=all_spikes)
view4 = SpikeTrainViewer(source=spike_source)
win.add_view(view4)
#show main window and run Qapp
win.show()


app.exec_()


/home/bapung/miniconda3/envs/data_analysis/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


KeyError: 'state'

In [4]:
str(filename)

'/data/Clustering/SleepDeprivation/RatN/Day1/RatN_Day1_2019-10-09_03-52-32.states.pkl'